In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml_tensorflow import GroceryML
from grocery_ml_core import GroceryMLCore
from hidden_layer_param_builder import HiddenLayerParamSetBuilder

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

def run_all_experiments(df, model_param_sets, output_dir):
    total = len(model_param_sets)
    print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
    for index, params in enumerate(model_param_sets, 1):
        print(f"Running Exp {index}/{total}...")
        groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)


try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


C:\Users\steve\source\repos\grocery-ml
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done


In [2]:

# quick test shot
layers = [
    { "units": 1, "activation": "relu" }
]
modelParamsList = []
modelParamsList.append({
    "trainParams": { "epochs": 1 },
    "buildParams": {
    "embedding_dim": 1,
    "layers": layers,
    "activation": "relu",
    "output_activation": "sigmoid",
    "optimizer": "adam",
    "learning_rate": 0.01,
    "loss": "binary_crossentropy",
    "metrics": "BinaryCrossentropy"
    }
})
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments(groceryML.training_df, modelParamsList, f"exp/keras/keras-1/test-shot-no-trust-{ts}")

run_all_experiments() when: 2026-01-14 01:12:28.022752  output_dir: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28
Running Exp 1/1...
Creating dir: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524
run_experiment()  exp_dir: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524  
run_experiment()  when: 2026-01-14 01:12:28.023899 params: {'epochs': 1}  
normalize_features()
train_model()
build_prediction_input() prediction_date=2026-01-14 01:12:29.090327
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:447: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
create_item_supply_level_feat()
normalize_features()
Running Model.Predict()
6/6 [==============================] - 0s 1ms/step
Exporting extra_dataframes:
grocery_ml_tensorflow.export_dataframes_to_excel()
Writing XLSX: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\normalized_training_df-e1_l1_ep1_sig_524.xlsx
   XLSX Done: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\normalized_training_df-e1_l1_ep1_sig_524.xlsx
Writing XLSX: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\predictions-e1_l1_ep1_sig_524.xlsx
   XLSX Done: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\predictions-e1_l1_ep1_sig_524.xlsx
[save_model] starting artifact save → exp/keras/keras-1/test-sho

INFO:tensorflow:Assets written to: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\model\assets


INFO:tensorflow:Assets written to: exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\model\assets


[save_model] saving model weights (separate file)
[save_model] all artifacts saved successfully → exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524\model
Saved experiment → exp/keras/keras-1/test-shot-no-trust-2026_01_14_01_12_28\e1_l1_ep1_sig_524


In [3]:
###########################################################

modelParamsList = []
layers = [
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 32, "activation": "relu" },
    
]
# lr_epoch_sets = [
#     { "learning_rate": 0.01,   "epochs": 10 },
#     { "learning_rate": 0.001,  "epochs": 40 },
#     { "learning_rate": 0.0001, "epochs": 120 }
# ]
output_activations = ["sigmoid", "linear"]
embDims = [300,500,700,1024]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for dim in embDims:
    for act in output_activations:
        modelParamsList.append({
            "trainParams": { "epochs": 50 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.0001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")

run_all_experiments() when: 2026-01-14 01:13:13.246338  output_dir: exp/keras/keras-1/2026_01_14_01_13_13
Running Exp 1/8...
Creating dir: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56
run_experiment()  exp_dir: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56  
run_experiment()  when: 2026-01-14 01:13:13.247339 params: {'epochs': 50}  
normalize_features()
train_model()
build_prediction_input() prediction_date=2026-01-14 01:13:30.465324
_build_combined_df()
_build_sources()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:447: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


creating target col: didBuy_target
insert_negative_samples()
compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
create_item_supply_level_feat()
normalize_features()
Running Model.Predict()
6/6 [==============================] - 0s 832us/step
Exporting extra_dataframes:
grocery_ml_tensorflow.export_dataframes_to_excel()
Writing XLSX: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56\normalized_training_df-e300_l128-128-64-32_ep50_sig_56.xlsx
   XLSX Done: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56\normalized_training_df-e300_l128-128-64-32_ep50_sig_56.xlsx
Writing XLSX: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56\predictions-e300_l128-128-64-32_ep50_sig_56.xlsx
   XLSX Done: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56\predictions-e3

INFO:tensorflow:Assets written to: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56\model\assets


INFO:tensorflow:Assets written to: exp/keras/keras-1/2026_01_14_01_13_13\e300_l128-128-64-32_ep50_sig_56\model\assets


...

compute_expected_gap_ewma()
create_item_supply_level_feat()
add_item_total_purchase_count_feat()
build_trip_interveral_feautres()
drop_rare_purchases()
validate_no_empty_columns()
self._build_combined_df() done
create_item_supply_level_feat()
normalize_features()
Running Model.Predict()
6/6 [==============================] - 0s 694us/step
Exporting extra_dataframes:
grocery_ml_tensorflow.export_dataframes_to_excel()
Writing XLSX: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\normalized_training_df-e500_l128-128-64-32_ep50_sig_13.xlsx
   XLSX Done: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\normalized_training_df-e500_l128-128-64-32_ep50_sig_13.xlsx
Writing XLSX: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\predictions-e500_l128-128-64-32_ep50_sig_13.xlsx
   XLSX Done: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\predictions-e500_l128-128-64-32_ep50_sig_13.xlsx
[save_model] starting arti

INFO:tensorflow:Assets written to: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\model\assets


INFO:tensorflow:Assets written to: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\model\assets


[save_model] saving model weights (separate file)
[save_model] all artifacts saved successfully → exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13\model
Saved experiment → exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_sig_13
Running Exp 4/8...
Creating dir: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_lin_15
run_experiment()  exp_dir: exp/keras/keras-1/2026_01_14_01_13_13\e500_l128-128-64-32_ep50_lin_15  
run_experiment()  when: 2026-01-14 01:16:46.689639 params: {'epochs': 50}  
normalize_features()
train_model()


KeyboardInterrupt: 

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    
]
output_activations = ["sigmoid", "linear"]
embDims = [1024,256,128,64,32,16,12]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    
]
output_activations = ["sigmoid", "linear",]
embDims = [1024,256,128,64,32,16,12]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 30 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.0001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },    
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    
    
]
output_activations = ["sigmoid", "linear",]
embDims = [1024,256,128,64,32,16,12]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 10 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.01,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },    
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    
    
]
output_activations = ["sigmoid", "linear", ]
embDims = [1024,256,128,64,32,16,12]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 10 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },    
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    { "units": 128, "activation": "relu" },
    
    
]
output_activations = ["sigmoid", "linear",]
embDims = [1024,256,128,64,32,16,12]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for act in output_activations:
    for dim in embDims:
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": layers,
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.0001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": metrics
            }
        })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments( groceryML.training_df, modelParamsList, f"exp/keras/keras-1/{ts}")